In [17]:
sqlFileToRun = "" # defaults to EXECUTE in .env

In [18]:
## run sql

from datetime import datetime
from uuid import uuid4
import hmac
import hashlib
import base64
import jaydebeapi
from dotenv import dotenv_values
import polars as pl
import re
env = dotenv_values(".env")

if sqlFileToRun.strip() != "":
    env['EXECUTE'] = sqlFileToRun.strip()
pl.Config.set_fmt_str_lengths(int(env['MAXCOLWIDTH']))


def clean_col(col):
    col = col.strip()

    if col.startswith("--") or "*" in col: #todo: load and pull the schema to give column names, query if missing from file
        return None

    if " AS " in col:
        col = col[col.index(" AS ") + 4:]
    if "." in col:
        col = col[col.index(".") + 1:]
    
    return col

# load query
with open(f"./queries/{env['EXECUTE']}.sql", "r") as file: #query
    query = "".join(file.readlines())

# load schema
if env['EXECUTE'] == "schema":
    query = query.replace("${DESCRIBE_TABLE}", env['DESCRIBE'])
    schema = { "table_qualifier": pl.Utf8, "table_name": pl.Utf8, "oa_table_userdata": pl.Utf8, "table_desc": pl.Utf8, "column_name": pl.Utf8, "type_name": pl.Utf8, "oa_length": pl.Int32, "oa_precision": pl.Int32, "oa_radix": pl.Int16, "oa_scale": pl.Int16, "oa_col_userdata": pl.Utf8, "column_desc": pl.Utf8, "pk_joinkey": pl.Utf8, "fk_joinkey": pl.Utf8 }
else:
    query = re.sub(r'--.*?\n', '', query)
    schema = query[query.index("SELECT ") + 7 : query.index(" FROM")].split(',')
    schema = list(filter(lambda col: col is not None, map(clean_col, schema)))

# generate password
timestamp = str(int(datetime.now().timestamp()))
nonce = uuid4().hex

base_string = "&".join((env["ACCOUNTID"], env["CONSUMERKEY"], env["TOKENID"], nonce, timestamp))
signature_key = "&".join((env["CONSUMERSECRET"], env["TOKENSECRET"]))

signature = base64.b64encode(hmac.new(
        signature_key.encode(),
        base_string.encode(),
        hashlib.sha256).digest()
    ).decode()

token_password = "&".join((base_string, signature, "HMAC-SHA256"))

# make connection
conn = jaydebeapi.connect(
    "com.netsuite.jdbc.openaccess.OpenAccessDriver",
    f'jdbc:ns://{env["ACCOUNTID"]}.connect.api.netsuite.com:1708;ServerDataSource=NetSuite2.com;encrypted=1;NegotiateSSLClose=false;tcpKeepAlive=true;CustomProperties=(AccountID={env["ACCOUNTID"]};RoleID={env["ROLEID"]})',
    [env["USER"], token_password],
    "./NQjc.jar"
    )

curs = conn.cursor()
curs.execute(query)


data=curs.fetchall()
df = pl.DataFrame(data, schema=schema)

if env['EXECUTE'] == "schema":
    df.write_parquet(f"./schema/{env['DESCRIBE']}.parquet")

df
